In [1]:
#import libraries
import pandas as pd
import xgboost as xgboost
import numpy as np

In [2]:
#import dataset
dataset = pd.read_csv('./dataset/bank-full.csv',sep=';')

In [3]:
#dataset.describe()
dataset.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [4]:
dataset.dtypes

age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
y            object
dtype: object

In [5]:
#isolate the x and y variables
y = dataset.iloc[:, -1].values
X = dataset._get_numeric_data()


In [6]:
#Split dataset into traininig and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2,random_state=1502)

In [7]:
#Transform y factor variables
y_train = np.where(y_train == 'yes', 1, 0)
y_test = np.where(y_test == 'yes', 1, 0)
np.mean(y_train) #11.5% is our baseline, we need to beat this
np.mean(y_test) #12.17%

0.12175163109587527

In [8]:
#Create xgbosst matrices
Train = xgboost.DMatrix(X_train, label = y_train)
Test = xgboost.DMatrix(X_test, label = y_test)

In [9]:
#Set parameters
parameters1 = {
    'learning_rate': 0.3,
    'max_depth': 2,
    'colsample_bytree': 1,
    'subsample': 1,
    'min_child_weight': 1,
    'gamma': 0,
    'random_state': 1502,
    'eval_metric': 'auc', 
    'objective': 'binary:logistic'
}

In [10]:
#run XGBoost
model = xgboost.train(params = parameters1, 
                      dtrain = Train, 
                      num_boost_round = 200,
                      evals = [(Test, 'Test')],
                      verbose_eval = 50
                      )

[0]	Test-auc:0.75049


[50]	Test-auc:0.87633
[100]	Test-auc:0.88087
[150]	Test-auc:0.88275
[199]	Test-auc:0.88375


In [11]:
#Predictions 
predictions1 =  model.predict(Test)
predictions1 = np.where(predictions1 > 0.5, 1, 0)

In [12]:
#Confusion matrix
# see lesson 27 for more details
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
confusion_matrix1 = confusion_matrix(y_test, predictions1)
print(confusion_matrix1)
report1 = classification_report(y_test, predictions1)
print(report1)

[[7690  252]
 [ 736  365]]
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      7942
           1       0.59      0.33      0.42      1101

    accuracy                           0.89      9043
   macro avg       0.75      0.65      0.68      9043
weighted avg       0.87      0.89      0.88      9043



In [13]:
#########################################################
# Transforming categorical variables into dummy variables
# Isolate the categorical variables
dataset_categorical = dataset.select_dtypes(exclude = "number")

# Transform categorical variables into dummy variables
dataset_categorical = pd.get_dummies(dataset_categorical, drop_first = True)

In [14]:
#joining numerical and categorical datasets
final_dataset = pd.concat([X, dataset_categorical], axis = 1)

In [15]:
#Getting names of columns
feature_columns = list(final_dataset.columns.values)
feature_columns = feature_columns[:-1]
#########################################################

In [16]:
#isolate the x and y variables Part 2
y = final_dataset.iloc[:, -1].values
X = final_dataset.iloc[:, :-1].values

#Split dataset into traininig and test set
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2,random_state=1502)

In [17]:
#Create xgbosst matrices Part 2
Train = xgboost.DMatrix(X_train, label = y_train, feature_names = feature_columns)
Test = xgboost.DMatrix(X_test, label = y_test, feature_names = feature_columns)

In [18]:
#Set parameters Part 2
parameters2 = {
    'learning_rate': 0.3,
    'max_depth': 2,
    'colsample_bytree': 1,
    'subsample': 1,
    'min_child_weight': 1,
    'gamma': 0,
    'random_state': 1502,
    'eval_metric': 'auc', 
    'objective': 'binary:logistic'
}

#run XGBoost Part 2
model2 = xgboost.train(params = parameters2, 
                      dtrain = Train, 
                      num_boost_round = 200,
                      evals = [(Test, 'Test')],
                      verbose_eval = 50
                      )

[0]	Test-auc:0.74351
[50]	Test-auc:0.91916
[100]	Test-auc:0.92513
[150]	Test-auc:0.92809
[199]	Test-auc:0.92905


In [19]:
#Predictions Part 2
predictions2 =  model2.predict(Test)
predictions2 = np.where(predictions2 > 0.5, 1, 0)

In [20]:
#Confusion matrix
# see lesson 27 for more details
confusion_matrix2 = confusion_matrix(y_test, predictions2)
print(confusion_matrix2)
report2 = classification_report(y_test, predictions2)
print(report2)

[[7676  266]
 [ 628  473]]
              precision    recall  f1-score   support

       False       0.92      0.97      0.94      7942
        True       0.64      0.43      0.51      1101

    accuracy                           0.90      9043
   macro avg       0.78      0.70      0.73      9043
weighted avg       0.89      0.90      0.89      9043



In [21]:
#########################################################

#Checking how many minicores we have
import multiprocessing
multiprocessing.cpu_count()

#Setting the cross validation parameters
from sklearn.model_selection import KFold
tune_control = KFold(n_splits = 5, 
                     shuffle = True, 
                     random_state = 1502).split(X = X_train, 
                                                y = y_train)


In [22]:
#Parameter tuning
#Set parameters Part 3
tune_grid = {
    'learning_rate': [0.05, 0.3],
    'max_depth': range(2,9,2),
    'colsample_bytree': [0.05, 1],
    'subsample': [1],
    'min_child_weight': [1],
    'gamma': [0],
    'random_state': [1502],
    "n_estimators": range(200, 2000, 200),
    "booster": ["gbtree"],
}

In [23]:
#State that we are doing a classification problem
from xgboost import XGBClassifier
classifier = XGBClassifier(objective = "binary:logistic")

In [24]:
#Cross Validation Assembly
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = tune_grid,
                           scoring = "roc_auc",
                           cv = tune_control,
                           n_jobs = 6,
                           verbose = 5)

In [25]:
#Setting evaluation parameters
evaluation_parameters = {
    "early_stopping_rounds": 100,
    "eval_metric": "auc",
    "eval_set": [(X_test, y_test)]
}

In [26]:
#Hyperparameter tuning and cross validation
tune_model = grid_search.fit(X = X_train, y = y_train, **evaluation_parameters)
grid_search.best_params_,grid_search.best_score_

Fitting 5 folds for each of 144 candidates, totalling 720 fits
[0]	validation_0-auc:0.84678
[1]	validation_0-auc:0.86974
[2]	validation_0-auc:0.87216
[3]	validation_0-auc:0.87349
[4]	validation_0-auc:0.88505
[5]	validation_0-auc:0.88833
[6]	validation_0-auc:0.89438
[7]	validation_0-auc:0.89527
[8]	validation_0-auc:0.89620
[9]	validation_0-auc:0.89654


c:\Users\carlo\miniconda3\envs\myenv\lib\site-packages\xgboost\sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
c:\Users\carlo\miniconda3\envs\myenv\lib\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[10]	validation_0-auc:0.89689
[11]	validation_0-auc:0.89728
[12]	validation_0-auc:0.89949
[13]	validation_0-auc:0.89947
[14]	validation_0-auc:0.89956
[15]	validation_0-auc:0.89962
[16]	validation_0-auc:0.89980
[17]	validation_0-auc:0.90006
[18]	validation_0-auc:0.90255
[19]	validation_0-auc:0.90401
[20]	validation_0-auc:0.90447
[21]	validation_0-auc:0.90515
[22]	validation_0-auc:0.90695
[23]	validation_0-auc:0.90708
[24]	validation_0-auc:0.90761
[25]	validation_0-auc:0.90837
[26]	validation_0-auc:0.90890
[27]	validation_0-auc:0.90958
[28]	validation_0-auc:0.90960
[29]	validation_0-auc:0.91062
[30]	validation_0-auc:0.91149
[31]	validation_0-auc:0.91212
[32]	validation_0-auc:0.91316
[33]	validation_0-auc:0.91360
[34]	validation_0-auc:0.91412
[35]	validation_0-auc:0.91436
[36]	validation_0-auc:0.91486
[37]	validation_0-auc:0.91517
[38]	validation_0-auc:0.91552
[39]	validation_0-auc:0.91571
[40]	validation_0-auc:0.91579
[41]	validation_0-auc:0.91604
[42]	validation_0-auc:0.91627
[43]	valid

({'booster': 'gbtree',
  'colsample_bytree': 1,
  'gamma': 0,
  'learning_rate': 0.05,
  'max_depth': 6,
  'min_child_weight': 1,
  'n_estimators': 600,
  'random_state': 1502,
  'subsample': 1},
 0.9338202687813076)